In [1]:
!python --version

Python 3.6.12 :: Anaconda, Inc.


In [ ]:
!pip install -q -U pip
!pip install -q numpy
!pip install -q pandas
!pip install -q ckiptagger
!pip install -q tqdm
!pip install -q tensorflow==1.14.0
!pip install -q ipywidgets

In [3]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

C:\Users\User\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dty

In [4]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [5]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
...,...,...,...
1795,1795,遊戲,LOL：麻辣香鍋韓服Rank不合成打野刀？電刀巨魔新套路連勝中
1796,1796,遊戲,穩住，我們能贏！因為我們擁有這種強大的力量
1797,1797,遊戲,騰訊是怎樣毀掉《鬥戰神》這款可能成神的作品的？
1798,1798,遊戲,LOL你不知道的黑科技打法！


In [6]:
df_test

,index,class,title
0,1800,體育,如果騎士火箭進入總決賽，誰的勝算大？
1,1801,體育,從個人競技狀態來看，三個階段的詹姆斯，哪個最強？
2,1802,體育,騎士總冠軍！地球人誰能阻擋詹姆斯？史上最佳就是他！打服所有人
3,1803,體育,詹姆斯絕殺，騎士3比0，猛龍懷疑人生
4,1804,體育,騎士和步行者戰成搶七險勝，而猛龍即將被橫掃，步行者跟猛龍的區別在哪裡？
...,...,...,...
595,2395,遊戲,如何玩好《陰陽師》？
596,2396,遊戲,網賺——玩遊戲網賺也不錯呦
597,2397,遊戲,迷你世界怎麼跳四段跳？
598,2398,遊戲,DNF5000罐子挑戰遠古遺願，玩家最後只說了兩個字「坑爹」


In [7]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [8]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [9]:
ws = WS('./data/')
pos = POS('./data/')

In [10]:
train_title_cuts = {}
#word_s, word_p = None, None
for index, title in tqdm(train_titles.items()):
    # YOUR CODE HERE
    #print(title)
    word_s = ws([title],
    sentence_segmentation =True)
    #print(word_s)
    
    word_p = pos(word_s)
    #print(word_p)
    #break
    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [11]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    # YOUR CODE HERE
    word_s = ws([title],sentence_segmentation =True)
    word_p = pos(word_s)
    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [12]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [13]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
# YOUR CODE HERE
i = 0
for lst_word_pos in train_title_cuts.values():
    for w, p in lst_word_pos:
        if w not in word2index:
            word2index[w] = i
            index2word[i] = w
            i += 1
# END YOUR CODE

In [14]:
word2index['溫暖']

1512

In [15]:
index2word[1512]

'溫暖'

In [16]:
def get_bow_vector(pairs, word2index):
    # YOUR CODE HERE
    vector = np.zeros(len(word2index))
    for w,p in pairs:
        if w in word2index:
            vector[word2index[w]] += 1

    return vector
    # END YOUR CODE

In [17]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 1., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [18]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['余小C', '雅桑克萊', '郎平', '約翰塞納', '2018-5-6', '招行', '重渡溝', '阿里雲', '帕克成', '后羿', '維韋杜詹霍', '賞月夜', '戈麥斯', '瑞和寶', '綠軍', '林斌', '切沃', '科爾', '烏迪內斯', '阿珂', '斯威', '沃爾瑪', '洛瑞', '郭春林', '查理芒格', '伯德', '劉詩雯', '勒布朗', 'NBA', '芒格', '石川佳純', '韓長賦', '西蒙斯', 'm15', '鎧單', '盧克肖', '里皮', 'S10', '保羅', '塔克', '英超', '扎哈維', '建聯', '唐汪鎮', '比格利亞', '姚振華', '克洛普', '德羅巴', '恆大', '上港', '朱之文', '何雯娜', '楊帆', '夢琪', '韋德', '奧拉朱旺遠', '喬丹', '馬克', '楊柳夏', '劉歡', '西游', '布茲德里克', '周評', '萬孚', '康德', '鄭眼', '朱婷', '小詹皇', '張掖篇', '木歷', '艷武漢', '李秋平', '朱嘯虎', '景泰石', '泰倫盧', '餘額寶', '梅西', '金英權', '曼聯', '賈躍亭', '德羅', '皇馬', '慧商', '華創', '卡拉斯科', '斯諾克', '阿木', '衛報', 'V10', '溫氏', '泰拳王西', '加威廉', '切爾西', '人民日報', '施泰納', '紫鑫', '殺里', '清華大學生', '盼保', '張大仙']
Na : ['中藥', '雨水', '湖怪', '薊馬', '傳奇', '可能', '電腦', '球童', '糧', '美味', '建築', '年齡', '獎勵', '蜂群', '好處', '金華人', '紅薯苗', '養殖戶', '醫藥', '回票', '迷宮', '潛質', '生態環境', '地球', '投資', '白鶴', '效果', '鯽魚塘頭價', '規範', '演講', '品種', '持續性', '魔鎧', '車', '姐', '自行車', '果農', '櫻花', '馬拉松賽', '銀聯卡', '髮力', '臍橙', '券商股', '網易商', '爆棚', '良機', '大蒜', '高溫', '陰影', '墨', 

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [37]:
def get_bow_vector_with_selection(pairs, word2index):
    # 根據以上列舉出來的文字以及詞性表，請列出想要排除的詞性
    # YOUR CODE HERE
    excluded_flags = [   
        'WHITESPACE','SPCHANGECATEGORY','SEMICOLONCATEGORY','QUESTIONCATEGORY','PERIODCATEGORY','PAUSECATEGORY',
        'PARENTHESISCATEGORY','EXCLAMATIONCATEGORY','ETCCATEGORY','DOTCATEGORY','DASHCATEGORY','COMMACATEGORY','COLONCATEGORY',
        'SHI','DE','V_2','T','P','Nh','Ng','Nf','Nd','Neu','Nes','Neqb','Neqa','Nep','I','D','Di','Dk','DM','Cbb','Cba','Cab','Caa'
    ]
    # END YOUR CODE
    vector = np.zeros(len(word2index))
    for word, pos in pairs:
        if word in word2index and pos not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [30]:
from numpy import linalg as la
def cosine_similarity(bow1, bow2):
    # YOUR CODE HERE
    similarity = np.dot(bow1,bow2) / (la.norm(bow1) * la.norm(bow2))
    # END YOUR CODE
    return similarity

In [38]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [32]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [33]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [39]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.04      , 0.00333333, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333])}

# Group mean vector: 測試

In [40]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [41]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 85, '遊戲': 8, '科技': 5, '農業': 4, '財經': 2, '旅遊': 2})
predict 財經 :  Counter({'財經': 73, '科技': 20, '農業': 7, '旅遊': 3, '遊戲': 2, '體育': 1})
predict 科技 :  Counter({'科技': 64, '財經': 15, '體育': 4, '旅遊': 4, '農業': 4, '遊戲': 2})
predict 旅遊 :  Counter({'旅遊': 78, '農業': 8, '體育': 3, '財經': 2, '科技': 1, '遊戲': 1})
predict 農業 :  Counter({'農業': 70, '旅遊': 8, '科技': 6, '財經': 5, '遊戲': 3})
predict 遊戲 :  Counter({'遊戲': 84, '體育': 6, '旅遊': 4, '科技': 3, '農業': 3, '財經': 1})
